In [1]:
from multiprocessing import get_context
import tensorflow as tf
import time
from decimal import Decimal, ROUND_HALF_UP
from scipy.optimize import brentq

import scipy.linalg as sl
def afm_est(Y, NF):

    n = Y.shape[0]
    p = Y.shape[1]
    
    # L'L normalization
    ev_ = sl.eigh(np.cov(Y))

    # Sort eigenvalues in descending order
    indx_ev = ev_[0].argsort()[::-1]
    # Get eigenvectors
    evec = ev_[1][:, indx_ev]

    # Determining Factors
    F = np.sqrt(n) * evec[:, 0:NF]

    # Factorloadings
    L = Y.T @ F/n

    """
    # F'F normalization
    ev_ = sl.eigh(np.cov(Y.T))

    # Sort eigenvalues in descending order
    indx_ev = ev_[0].argsort()[::-1]
    # Get eigenvectors
    evec = ev_[1][:, indx_ev]

    # Determining Factors
    L = np.sqrt(p) * evec[:, 0:NF]
    # Factorloadings
    F = Y @ L/p
    """
    resd = Y - F @ L.T

    ret_ = {'F': F, 'L': L, 'resd': resd}

    return ret_

def nf_bn(Y, nf_max):

    n = Y.shape[0]
    p = Y.shape[1]

    IC1 = np.empty((nf_max, 1))
    IC1[:] = np.nan
    IC2 = np.empty((nf_max, 1))
    IC2[:] = np.nan
    IC3 = np.empty((nf_max, 1))
    IC3[:] = np.nan

    for ii in range(1, nf_max+1):
        ret_afm = afm_est(Y, ii)

        V = np.mean(ret_afm['resd']**2)
        
        #Information criteria
        #IC1
        IC1[ii-1, 0] = np.log(V) + ii * ((p+n)/(p*n) * np.log((p*n)/(p+n)))
        #IC2
        IC2[ii-1, 0] = np.log(V) + ii * ((p+n)/(p*n) * np.log(min(n,p)))
        #IC3
        IC3[ii-1, 0] = np.log(V) + ii * (np.log(min(n,p)) / (min(n,p)))
    
    ICs = np.empty((3, 2))
    ICs[:] = np.nan

    ICs[0, 0] = IC1.argmin()
    ICs[1, 0] = IC2.argmin()
    ICs[2, 0] = IC3.argmin()

    ICs[0, 1] = IC1[int(ICs[0, 0])]
    ICs[1, 1] = IC2[int(ICs[1, 0])]
    ICs[2, 1] = IC3[int(ICs[2, 0])]

    ret_ = {'num_f': ICs[:,0], 'ICs': ICs[:,1]}

    return ret_

def soft_t(z, a):
  t1 = np.sign(z)
  b = np.abs(z) - a
  t2 = b * (b >= 0)
  z_t = t1 * t2
  return z_t

def cov_e_poet(resd, C, N, T):
    rate_thres = 1/np.sqrt(N) + np.sqrt((np.log(N))/T)
    # lam = rate_thres * C * np.ones(shape=(N,N))
    
    sig_e_samp = np.cov(resd.T)
    
    
    thet_par = np.empty((N, N))
    thet_par[:] = np.nan
    
    for ii in range(0, N):
        for jj in range(0, N):
            thet_par[ii, jj] = np.mean((resd[:, ii] * resd[:, jj] - sig_e_samp[ii, jj])**2)
    
    lam = rate_thres * C * np.sqrt(thet_par)
    
    """
    sig_e_diag=np.diag(np.sqrt(np.diag(sig_e_samp)))
    R = np.linalg.inv(sig_e_diag) @ sig_e_samp @ np.linalg.inv(sig_e_diag); 
    M = soft_t(R, lam)
    np.fill_diagonal(M, 1)
    sig_e_hat = sig_e_diag @ M @ sig_e_diag
    """

    sig_e_diag = np.diag(sig_e_samp)
    sig_e_hat = soft_t(sig_e_samp, lam)
    np.fill_diagonal(sig_e_hat, sig_e_diag)

    return sig_e_hat

def poet(Y_star,NF_max):
    n = Y_star.shape[0]
    p = Y_star.shape[1]
    
    num_f = nf_bn(Y_star, NF_max)['num_f']
    num_f = int(num_f[0])
    est_afm = afm_est(Y_star, num_f)

    F = est_afm['F']
    L = est_afm['L']
    resd = est_afm['resd']
    sigma_u_hat = cov_e_poet(resd, 2, p, n)
    sigma_u_hat_inv = np.linalg.inv(sigma_u_hat)
    
    A=L.T @ sigma_u_hat_inv @ L
    I=np.eye(A.shape[0])
    
    Theta_hat = sigma_u_hat_inv - sigma_u_hat_inv @ L @ np.linalg.inv(I+A) @ L.T @ sigma_u_hat_inv
    return Theta_hat


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star

def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Solves the constrained optimization:
    min w' Sigma w  subject to  w' mu = target_return  and  w' 1 = 1
    
    Solution uses Lagrange multipliers with two constraints.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    long_only : bool
        If True, falls back to GMV if MV produces negative weights
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p  # 1' Theta 1
    B = ones_p @ Theta_hat @ mu       # 1' Theta mu  
    C = mu @ Theta_hat @ mu           # mu' Theta mu
    D = A * C - B * B                  # Determinant
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        print('SINGULARITY')
        # System is singular, use GMV instead
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights: w = lambda1 * Theta^{-1} 1 + lambda2 * Theta^{-1} mu
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star

def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    The maximum Sharpe ratio portfolio solves:
    max (w' mu) / sqrt(w' Sigma w)
    
    Solution (when mu represents excess returns):
    w ∝ Sigma^{-1} mu = Theta mu
    
    Then normalize so that sum(w) = 1.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights: w ∝ Theta mu
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        print('WARNING: Weight sum near zero, returning equal weights')
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Assuming DNN_FM_main, gmv_weights, mv_weights, msr_weights are imported
# from your existing modules

def train_logistic_regression(df, train_start, train_end, features=['mom12m', 'mve', 'bm']):
    """
    Train logistic regression on historical data to predict positive returns.
    """
    train_df = df[(df['datadate'] >= train_start) & (df['datadate'] <= train_end)].copy()
    
    # Create binary target: 1 if positive return, 0 otherwise
    train_df['target'] = (train_df['ret_fwd_1'] > 0).astype(int)
    
    # Remove rows with missing values
    train_df = train_df.dropna(subset=features + ['target'])
    
    # Prepare features
    X_train = train_df[features]
    y_train = train_df['target']
    
    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Train logistic regression
    log_reg = LogisticRegression(random_state=42, max_iter=1000)
    log_reg.fit(X_train_scaled, y_train)
    
    print(f"  Training samples: {len(train_df)}")
    print(f"  Training accuracy: {log_reg.score(X_train_scaled, y_train):.4f}")
    
    return log_reg, scaler


def select_stocks_with_logistic(df, predict_date, log_reg, scaler, 
                                 features=['mom12m', 'mve', 'bm'],
                                 method='top_n', n_stocks=100, threshold=0.5):
    """
    Use trained logistic regression to select stocks for a given date.
    """
    predict_df = df[df['datadate'] == predict_date].copy()
    predict_df = predict_df.dropna(subset=features)
    
    if len(predict_df) == 0:
        print(f"  ⚠ No stocks with complete data on {predict_date}")
        return []
    
    # Prepare features for prediction
    X_predict = predict_df[features]
    X_predict_scaled = scaler.transform(X_predict)
    
    # Generate buy probabilities
    predict_df['buy_probability'] = log_reg.predict_proba(X_predict_scaled)[:, 1]
    
    # Select stocks based on method
    if method == 'top_n':
        selected_df = predict_df.nlargest(n_stocks, 'buy_probability')
    elif method == 'top_and_bottom':
        top_n = predict_df.nlargest(n_stocks, 'buy_probability')
        bottom_n = predict_df.nsmallest(n_stocks, 'buy_probability')
        selected_df = pd.concat([top_n, bottom_n])
    elif method == 'threshold':
        selected_df = predict_df[predict_df['buy_probability'] >= threshold]
    else:
        raise ValueError(f"Unknown method: {method}")
    
    selected_permnos = selected_df['permno'].tolist()
    
    print(f"  Stocks evaluated: {len(predict_df)}")
    print(f"  Stocks selected: {len(selected_permnos)}")
    print(f"  Buy probability range: [{predict_df['buy_probability'].min():.4f}, "
          f"{predict_df['buy_probability'].max():.4f}]")
    
    return selected_permnos


def integrated_backtest(df, 
                        test_start_date='2020-01-31', 
                        test_end_date='2024-11-30',
                        logistic_train_years=15,
                        logistic_features=['mom12m', 'mve', 'bm'],
                        stock_selection_method='top_n',
                        n_stocks=100,
                        lookback_window=180,
                        transaction_cost=0.001,
                        data_factor=None,
                        dnn_architecture=5,
                        dnn_const_err_cov=2.5,
                        mv_target_return=0.01,
                        verbose=True):
    """
    Integrated backtest combining logistic regression stock selection with DNN-FM optimization.
    
    Annual workflow:
    1. Train logistic regression on past N years (every January)
    2. Select stocks based on buy probability
    3. Run DNN-FM on selected stocks monthly until next retrain
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1, mom12m, mve, bm
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    logistic_train_years : int
        Number of years to use for training logistic regression (default: 15)
    logistic_features : list
        Features for logistic regression (default: ['mom12m', 'mve', 'bm'])
    stock_selection_method : str
        'top_n', 'threshold', or 'top_and_bottom' (default: 'top_n')
    n_stocks : int
        Number of stocks to select (default: 100)
    lookback_window : int
        Number of months in rolling training window for DNN-FM (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.001 = 10 bps)
    data_factor : pd.DataFrame
        Factor data for DNN-FM model
    dnn_architecture : int
        Architecture choice for DNN-FM (default: 5)
    dnn_const_err_cov : float
        Error covariance constant for DNN-FM (default: 2.5)
    mv_target_return : float
        Target return for MV portfolio (default: 0.01 = 1% monthly)
    verbose : bool
        If True, prints detailed log at each time step
    
    Returns:
    --------
    results_dict : dict
        Dictionary with keys 'gmv', 'mv', 'msr', each containing:
        {'results_df': DataFrame, 'metrics': dict}
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback window")
    
    # Extract years that need logistic regression retraining
    start_year = test_start_dt.year
    end_year = test_end_dt.year
    test_years = list(range(start_year, end_year + 1))
    
    # Portfolio types to compute
    portfolio_types = ['gmv', 'mv', 'msr']
    
    # Storage for each portfolio type
    results_storage = {ptype: {
        'returns': [],
        'dates': [],
        'weights_list': [],
        'turnover_list': [],
        'gross_returns': [],
        'prev_weights_dict': {},
        'prev_oos_returns_dict': {},
        'prev_gross_return': 0.0
    } for ptype in portfolio_types}
    
    # Track current stock universe
    current_permnos = []
    
    if verbose:
        print("="*70)
        print("INTEGRATED BACKTEST: LOGISTIC REGRESSION + DNN-FM")
        print(f"Test Period: {test_start_date} to {test_end_date}")
        print("="*70)
    
    # --- 2. Annual Logistic Retraining Loop ---
    for year in test_years:
        # Retrain logistic regression in January
        retrain_date = pd.to_datetime(f'{year}-01-31')
        
        if retrain_date not in all_dates:
            print(f"\n⚠ Warning: {retrain_date} not in dataset, skipping year {year}")
            continue
        
        if verbose:
            print(f"\n{'='*70}")
            print(f"YEAR {year}: RETRAINING LOGISTIC REGRESSION")
            print(f"{'='*70}")
        
        # Define training window for logistic regression
        train_end = pd.to_datetime(f'{year-1}-12-31')
        train_start = pd.to_datetime(f'{year - logistic_train_years}-01-31')
        
        if verbose:
            print(f"Logistic training period: {train_start.strftime('%Y-%m-%d')} to "
                  f"{train_end.strftime('%Y-%m-%d')}")
        
        # Train logistic regression
        log_reg, scaler = train_logistic_regression(
            df, train_start, train_end, features=logistic_features
        )
        
        # Select stocks for this year
        current_permnos = select_stocks_with_logistic(
            df, retrain_date, log_reg, scaler,
            features=logistic_features,
            method=stock_selection_method,
            n_stocks=n_stocks
        )
        
        if len(current_permnos) == 0:
            print(f"  ⚠ No stocks selected for {year}, skipping")
            continue
        
        # Determine date range for this year's strategy
        year_start_date = retrain_date
        
        if year == end_year:
            year_end_date = test_end_dt
        else:
            year_end_date = pd.to_datetime(f'{year}-12-31')
            if year_end_date not in all_dates:
                year_dates = [d for d in all_dates if d.year == year]
                year_end_date = max(year_dates) if year_dates else year_start_date
        
        # For the first year, respect test_start_date
        if year == start_year and test_start_dt > year_start_date:
            year_start_date = test_start_dt
        
        try:
            year_start_idx = all_dates.index(year_start_date)
            year_end_idx = all_dates.index(year_end_date)
        except ValueError as e:
            print(f"  ⚠ Date error: {e}")
            continue
        
        if verbose:
            print(f"\nRunning monthly rebalancing from {year_start_date.strftime('%Y-%m-%d')} "
                  f"to {year_end_date.strftime('%Y-%m-%d')}")
            print(f"{'='*70}")
        
        # --- 3. Monthly Loop for DNN-FM ---
        for t in range(year_start_idx, year_end_idx + 1):
            current_date = all_dates[t]
            
            if t < lookback_window:
                if verbose:
                    print(f"\n[{current_date.strftime('%Y-%m-%d')}] "
                          f"Skipping: insufficient lookback")
                continue
            
            # Define training window
            window_start_date = all_dates[t - lookback_window]
            window_end_date = all_dates[t - 1]
            
            # Filter data to selected stocks only
            train_data = df[
                (df['datadate'] >= window_start_date) & 
                (df['datadate'] <= window_end_date) &
                (df['permno'].isin(current_permnos))
            ]
            
            # Pivot returns
            returns_pivot = train_data.pivot(
                index='datadate', columns='permno', values='ret_fwd_1'
            )
            
            # Reindex to ensure all dates
            window_dates = all_dates[t - lookback_window : t]
            returns_pivot = returns_pivot.reindex(index=window_dates)
            
            # Get factor data for this window
            train_factor = data_factor.loc[window_start_date : window_end_date]
            
            # Filter assets with any NaNs
            nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
            filtered_pivot = returns_pivot.drop(columns=nan_assets)
            
            current_assets = filtered_pivot.columns.tolist()
            Y = filtered_pivot.values
            n_train, p_current = Y.shape
            
            if verbose:
                month_num = t - year_start_idx + 1
                print(f"\n[Month {month_num}] {current_date.strftime('%Y-%m-%d')}")
                print(f"  Assets: {p_current}/{len(current_permnos)} with complete data")
            
            # Check validity
            if n_train < lookback_window or p_current < 2:
                if verbose:
                    print(f"  ⚠ Insufficient data, using previous weights")
                
                new_weights_dict = {ptype: results_storage[ptype]['prev_weights_dict'].copy() 
                                   for ptype in portfolio_types}
            else:
                try:
                    # Demean
                    Y_bar = Y.mean(axis=0)
                    Y_star = Y - Y_bar
                    
                    # Run DNN-FM
                    if verbose:
                        print(f"  Running DNN-FM...")
                    F = train_factor.values.astype(float)
                    Theta_hat = poet(Y_star, 8)
                    
                    # Compute weights for each portfolio type
                    new_weights_dict = {}
                    
                    if verbose:
                        print(f"  Computing portfolio weights...")
                    
                    w_gmv = gmv_weights(Theta_hat)
                    new_weights_dict['gmv'] = {
                        asset: w_gmv[i] for i, asset in enumerate(current_assets)
                    }
                    
                    w_mv = mv_weights(Theta_hat, Y_bar, target_return=mv_target_return)
                    new_weights_dict['mv'] = {
                        asset: w_mv[i] for i, asset in enumerate(current_assets)
                    }
                    
                    w_msr = msr_weights(Theta_hat, Y_bar)
                    new_weights_dict['msr'] = {
                        asset: w_msr[i] for i, asset in enumerate(current_assets)
                    }
                    
                    if verbose:
                        print(f"  ✓ DNN-FM completed for GMV, MV, MSR")
                    
                except Exception as e:
                    if verbose:
                        print(f"  ✗ Error: {e}")
                    new_weights_dict = {ptype: results_storage[ptype]['prev_weights_dict'].copy() 
                                       for ptype in portfolio_types}
            
            # Get OOS returns (common for all portfolio types)
            oos_data = df[df['datadate'] == current_date]
            oos_returns_series = oos_data.set_index('permno')['ret_fwd_1'].dropna()
            oos_returns_dict = oos_returns_series.to_dict()
            
            # --- 4. Process Each Portfolio Type ---
            for ptype in portfolio_types:
                # Normalize weights
                weights = new_weights_dict[ptype]
                weight_sum = sum(weights.values())
                if weight_sum > 1e-10:
                    weights = {k: v/weight_sum for k, v in weights.items()}
                else:
                    weights = results_storage[ptype]['prev_weights_dict'].copy()
                
                # Common assets
                common_assets = set(weights.keys()) & set(oos_returns_dict.keys())
                
                if len(common_assets) == 0:
                    continue
                
                # Filter and renormalize
                common_weights = {a: weights[a] for a in common_assets}
                common_weight_sum = sum(common_weights.values())
                if common_weight_sum > 1e-10:
                    common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
                else:
                    continue
                
                # Gross return
                gross_return = sum(
                    common_weights[a] * oos_returns_dict[a] for a in common_assets
                )
                
                if np.isnan(gross_return) or np.isinf(gross_return):
                    continue
                
                # Transaction costs
                prev_weights = results_storage[ptype]['prev_weights_dict']
                prev_oos_returns = results_storage[ptype]['prev_oos_returns_dict']
                prev_gross_ret = results_storage[ptype]['prev_gross_return']
                
                if len(prev_weights) > 0:
                    adjusted_prev = {}
                    for asset, prev_w in prev_weights.items():
                        if asset in prev_oos_returns:
                            prev_r = prev_oos_returns[asset]
                            if abs(1 + prev_gross_ret) > 1e-6:
                                adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_ret)
                            else:
                                adjusted_prev[asset] = 0.0
                        else:
                            if abs(1 + prev_gross_ret) > 1e-6:
                                adjusted_prev[asset] = prev_w / (1 + prev_gross_ret)
                            else:
                                adjusted_prev[asset] = 0.0
                    
                    all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
                    turnover = sum(
                        abs(common_weights.get(a, 0.0) - adjusted_prev.get(a, 0.0))
                        for a in all_assets
                    )
                    tc = transaction_cost * (1 + gross_return) * turnover
                else:
                    turnover = sum(abs(w) for w in common_weights.values())
                    tc = transaction_cost * (1 + gross_return) * turnover
                
                net_return = gross_return - tc
                
                # Store results
                results_storage[ptype]['returns'].append(net_return)
                results_storage[ptype]['dates'].append(current_date)
                results_storage[ptype]['weights_list'].append(common_weights.copy())
                results_storage[ptype]['turnover_list'].append(turnover)
                results_storage[ptype]['gross_returns'].append(gross_return)
                
                # Update state
                results_storage[ptype]['prev_weights_dict'] = common_weights.copy()
                results_storage[ptype]['prev_oos_returns_dict'] = {a: oos_returns_dict[a] for a in common_assets}
                results_storage[ptype]['prev_gross_return'] = gross_return
            
            if verbose:
                # Print summary for all portfolios
                print(f"  Portfolio Returns:")
                for ptype in portfolio_types:
                    if len(results_storage[ptype]['returns']) > 0:
                        last_idx = len(results_storage[ptype]['returns']) - 1
                        gross_ret = results_storage[ptype]['gross_returns'][last_idx]
                        net_ret = results_storage[ptype]['returns'][last_idx]
                        to = results_storage[ptype]['turnover_list'][last_idx]
                        tc = gross_ret - net_ret
                        print(f"    {ptype.upper()}: Gross={gross_ret:>7.4f} | TO={to:>5.3f} | "
                              f"TC={tc:>7.5f} | Net={net_ret:>7.4f}")
    
    if verbose:
        print("\n" + "="*70)
        print("BACKTEST COMPLETE")
        print("="*70)
    
    # --- 5. Compile Results ---
    results_dict = {}
    
    for ptype in portfolio_types:
        portfolio_returns = results_storage[ptype]['returns']
        portfolio_dates = results_storage[ptype]['dates']
        portfolio_turnover_list = results_storage[ptype]['turnover_list']
        portfolio_gross_returns = results_storage[ptype]['gross_returns']
        
        if len(portfolio_returns) == 0:
            results_dict[ptype] = {
                'results_df': pd.DataFrame(),
                'metrics': {}
            }
            continue
        
        results_df = pd.DataFrame({
            'date': portfolio_dates,
            'portfolio_return': portfolio_returns,
            'portfolio_gross_return': portfolio_gross_returns,
            'portfolio_turnover': portfolio_turnover_list
        })
        results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
        
        # Compute metrics
        mean_return = np.mean(portfolio_returns)
        variance = np.var(portfolio_returns, ddof=1)
        sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
        
        annual_return = mean_return * 12
        annual_volatility = np.sqrt(variance * 12)
        annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
        
        overall_metrics = {
            'mean_return': mean_return,
            'variance': variance,
            'sharpe_ratio': sharpe_ratio,
            'annual_return': annual_return,
            'annual_volatility': annual_volatility,
            'annual_sharpe_ratio': annual_sharpe,
            'total_return': results_df['cumulative_return'].iloc[-1],
            'avg_turnover': np.mean(portfolio_turnover_list),
            'n_periods': len(portfolio_returns)
        }
        
        results_dict[ptype] = {
            'results_df': results_df,
            'metrics': overall_metrics
        }
    
    return results_dict

In [3]:
df = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = (df.groupby('permno')['ret_excess'].shift(-1) )

In [4]:
data_f=pd.read_csv('F-F_Research_Data_Factors.csv',sep=',')
data_f['Date']=pd.to_datetime(data_f['Date'], format="%Y%m")
data_f['Date']=data_f['Date']+pd.offsets.MonthEnd(0)
data_f = data_f.set_index('Date')
data_f = data_f[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

results = integrated_backtest(
    df,
    test_start_date='2020-01-31',
    test_end_date='2024-04-30',
    logistic_train_years=15,
    stock_selection_method='top_and_bottom',
    n_stocks=50,
    lookback_window=180,
    transaction_cost=0.001,
    data_factor=data_f,
    verbose=True
)

# Access results for each portfolio
gmv_results = results['gmv']['results_df']
gmv_metrics = results['gmv']['metrics']

mv_results = results['mv']['results_df']
mv_metrics = results['mv']['metrics']

msr_results = results['msr']['results_df']
msr_metrics = results['msr']['metrics']

# Compare Sharpe ratios
print(f"GMV Sharpe: {gmv_metrics['annual_sharpe_ratio']:.4f}")
print(f"MV Sharpe:  {mv_metrics['annual_sharpe_ratio']:.4f}")
print(f"MSR Sharpe: {msr_metrics['annual_sharpe_ratio']:.4f}")

INTEGRATED BACKTEST: LOGISTIC REGRESSION + DNN-FM
Test Period: 2020-01-31 to 2024-04-30

YEAR 2020: RETRAINING LOGISTIC REGRESSION
Logistic training period: 2005-01-31 to 2019-12-31
  Training samples: 89793
  Training accuracy: 0.5590
  Stocks evaluated: 497
  Stocks selected: 100
  Buy probability range: [0.5322, 0.5826]

Running monthly rebalancing from 2020-01-31 to 2020-12-31

[Month 1] 2020-01-31
  Assets: 64/100 with complete data
  Running DNN-FM...
  Computing portfolio weights...
  ✓ DNN-FM completed for GMV, MV, MSR
  Portfolio Returns:
    GMV: Gross=-0.0829 | TO=1.241 | TC=0.00114 | Net=-0.0841
    MV: Gross=-0.0841 | TO=1.231 | TC=0.00113 | Net=-0.0852
    MSR: Gross=-0.0694 | TO=1.488 | TC=0.00139 | Net=-0.0707

[Month 2] 2020-02-29
  Assets: 64/100 with complete data
  Running DNN-FM...
  Computing portfolio weights...
  ✓ DNN-FM completed for GMV, MV, MSR
  Portfolio Returns:
    GMV: Gross=-0.0581 | TO=0.033 | TC=0.00003 | Net=-0.0581
    MV: Gross=-0.0558 | TO=0.100 

In [5]:
print(f"\n GMV")
print(f"Annualized Sharpe Ratio: {gmv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {gmv_metrics['mean_return']*12:.4f}")
print(f"Variance: {gmv_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {gmv_metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"Annualized Sharpe Ratio: {mv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {mv_metrics['mean_return']*12:.4f}")
print(f"Variance: {mv_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {mv_metrics['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"Annualized Sharpe Ratio: {msr_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {msr_metrics['mean_return']*12:.4f}")
print(f"Variance: {msr_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {msr_metrics['avg_turnover']:.4f}")


 GMV
Annualized Sharpe Ratio: 0.4265
Mean Return: 0.0636
Variance: 0.0222
Avg Turnover: 0.2189

 MV
Annualized Sharpe Ratio: 0.4503
Mean Return: 0.0660
Variance: 0.0215
Avg Turnover: 0.2412

 MSR
Annualized Sharpe Ratio: 0.4456
Mean Return: 0.0703
Variance: 0.0249
Avg Turnover: 0.2761
